In [1]:
# Jupyter magic extensions that makes our live easier
%load_ext autoreload
%autoreload 2

In [26]:
from src.models.train_pipeline import train_pipeline
from src.models.models import SklearnAdaBoostModel
from src.data.make_dataset import PandasDatasetLoader

dataset_path = "data/raw"
labels_path = "data/labels/"

train_pipeline(PandasDatasetLoader(dataset_path, labels_path), SklearnAdaBoostModel())

Resultado AUC:  {'1110': 0.8333333333333334} 

Reporte:  {'1110': '              precision    recall  f1-score   support\n\n           0       1.00      0.67      0.80         3\n           1       0.67      1.00      0.80         2\n\n    accuracy                           0.80         5\n   macro avg       0.83      0.83      0.80         5\nweighted avg       0.87      0.80      0.80         5\n'} 



In [3]:
import os

print(os.getcwd())

os.chdir("..")

/home/jmjl/PycharmProjects/epilepsia_challenge/notebooks


In [4]:
import os

print(os.getcwd())

c:\Users\rober\Documents\Epilepsy_Challenge\epilepsia_challenge
